## Sentinel-2 Image (Dataset Availability: 2015-06-27 to Present)
### [Sentinel-2 Earth Engine Documentation](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED)

In [1]:
# Importing the earth engine library and geemap package
import ee
import geemap

In [2]:
# Adding an interactive map.
Map = geemap.Map(basemap='ROADMAP', center=[24, 91], zoom=7, height=600, draw = True)

# Adding the TERRAIN basemaps.
Map.add_basemap("TERRAIN")
Map

Map(center=[24, 91], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [3]:
# Taking a draw feature from the map.
feature = Map.draw_last_feature

# If drawing does not exist then taking a polygon as the default version.
if feature is None:
    geom = ee.Geometry.Polygon([[[91.78481923595828, 21.95584672623341],
                                 [91.78481923595828, 21.700881280747268],
                                 [91.93176137463016, 21.700881280747268],
                                 [91.93176137463016, 21.95584672623341]]])
    feature = ee.Feature(geom, {})

# Define the AOI.
AOI = feature.geometry()

In [13]:
# Function to mask clouds using the Sentinel-2 SCL band.
def S2CloudMaskSCL(img):
  qa = img.select('SCL')
  mask_band = qa.eq(3).Or(qa.eq(8)).Or(qa.eq(9)).Or(qa.eq(10)).Or(qa.eq(11))
  mask = mask_band.eq(0)
  # Return the masked and scaled data.
  return img.updateMask(mask)

def ESAcloudMask(img):
    ''' Sentinel-2 Bits 10 & 11 are clouds & cirrus, so set to 0. '''
    qa = img.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(clear)

In [36]:
start_date = '2023-01-01'
end_date = '2023-12-31'

In [33]:
#  Filter Sentinel2 collection and apply the cloud mask.
clound_masked = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                      .filterDate(start_date, end_date)
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                      .filterBounds(AOI)
                      .map(S2CloudMaskSCL)
                      .map(ESAcloudMask))

In [34]:
# Without cloud masking
sentinel_2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                      .filterDate(start_date, end_date)
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                      .filterBounds(AOI))

In [35]:
print ('Number of Sentinel-2 images:', sentinel_2_collection.size().getInfo())

Number of Sentinel-2 images: 0


In [18]:
# Specify visualization parameters.
vis_natural = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0.0,
    'max': 3000,
}

vis_nir = {
    'bands': ['B12', 'B8A', 'B4'],
    'min': 0.0,
    'max': 3000,
}

# Generate Earth Engine layers.
left_layer = geemap.ee_tile_layer(sentinel_2_collection.first(), vis_natural,  'with cloud')
right_layer = geemap.ee_tile_layer(clound_masked.first(), vis_nir, 'without cloud')

# Create a split-panel map.
Map_S2 = geemap.Map(height=600)
Map_S2.split_map(left_layer, right_layer)
Map_S2.centerObject(sentinel_2_collection, 8)
Map_S2

Map(center=[22.099105462702703, 91.64545333544221], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [19]:
# Creating a cloud masked median image
S2_masked_median = clound_masked.median().clip(AOI)

# Vis paramateres
vis_s2 = {"min":0.0, "max":3000,"bands": ['B8','B4','B3']}

Map_all = geemap.Map(height=500)
Map_all.centerObject(S2_masked_median, 9)
Map_all.addLayer(S2_masked_median, vis_s2, 'Sentinel-2 median')
Map_all

Map(center=[21.82834239157615, 91.85829030529537], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
# Export the data to your google drive
task = ee.batch.Export.image.toDrive(
            image = S2_masked_median.select(["B8","B4","B3"]).uint16(),
            description = 'S2_Median_2023',
            folder = 'Sentinel_Image',
            scale = 10,
            maxPixels = 10e12,
            fileDimensions= 256*2*100,
            shardSize= 256*2,
            formatOptions = {'cloudOptimized': True})

task.start()